In [ ]:
import fastf1 as ff1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# USE TO GET THE ROUUND NUMBERS
schedule = ff1.get_event_schedule(2021)
# print(schedule)
# print(schedule.columns)
print(schedule.loc[:,['RoundNumber', 'Country', 'Location']])

In [ ]:
#LOAD DATA

YEAR = 2021
RACE = 'Great Britain'
SESSION = 'Q'
print(f"\nLoading {YEAR} {RACE} GP - {SESSION} session...")
session = ff1.get_session(YEAR,RACE,SESSION)
session.load()

circuit_info = session.event
print(f"\n{'='*60}")
print(f"CIRCUIT INFORMATION")
print(f"{'='*60}")
print(f"Event: {circuit_info['EventName']}")
print(f"Location: {circuit_info['Location']}")
print(f"Country: {circuit_info['Country']}")
print(f"Circuit: {circuit_info['OfficialEventName']}")
print(f"Date: {circuit_info['EventDate']}")
print(f"{'='*60}\n")

In [ ]:
# SELECT DRIVERS

driver1 = 'VER'
driver2 = 'HAM'

driver1_fastest = session.laps.pick_driver(driver1).pick_fastest()
driver2_fastest = session.laps.pick_driver(driver2).pick_fastest()
driver1_name = driver1_fastest['Driver']
driver2_name = driver2_fastest['Driver']


In [ ]:
# LAP TIMES TABLE

driver1_laps = session.laps.pick_driver(driver1)
driver2_laps = session.laps.pick_driver(driver2)

lap_data = []
for i, lap in enumerate(driver1_laps.iterlaps(),1):
    lap_data.append({
        'Lap': i,
        f'{driver1} Time': lap[1]['LapTime'],
        f'{driver1} Fastest': '★' if lap[1]['LapTime'] == driver1_fastest['LapTime'] else ''
    })

for i, lap in enumerate(driver2_laps.iterlaps(),1):
    if i <= len(lap_data):
        lap_data[i-1][f'{driver2} Time'] = lap[1]['LapTime']
        lap_data[i-1][f'{driver2} Fastest'] = '★' if lap[1]['LapTime'] == driver2_fastest['LapTime'] else ''
    else:
        lap_data.append({
            'Lap': i,
            f'{driver1} Time': None,
            f'{driver1} Fastest': '',
            f'{driver2} Time': lap[1]['LapTime'],
            f'{driver2} Fastest': '★' if lap[1]['LapTime'] == driver2_fastest['LapTime'] else ''
        })
lap_times_df = pd.DataFrame(lap_data)
print("\n" + "="*80)
print(f"LAP TIMES COMPARISON - {driver1} vs {driver2}")
print("="*80)
print(f"★ indicates the fastest lap used in the animation")
print("-"*80)
print(lap_times_df.to_string(index=False))
print("="*80)

print(f"\n{driver1} Fastest Lap: {driver1_fastest['LapTime']}")
print(f"{driver2} Fastest Lap: {driver2_fastest['LapTime']}")

In [ ]:
# GET TELEMETRY

driver1_tel = driver1_fastest.get_telemetry()
driver2_tel = driver2_fastest.get_telemetry()
SAMPLE_RATE = 3
driver1_tel = driver1_tel.iloc[::SAMPLE_RATE].reset_index(drop=True)
driver2_tel = driver2_tel.iloc[::SAMPLE_RATE].reset_index(drop=True)

driver1_x = driver1_tel['X'].values
driver1_y = driver1_tel['Y'].values
driver2_x = driver2_tel['X'].values
driver2_y = driver2_tel['Y'].values
driver1_time = driver1_tel['Time'].dt.total_seconds().values
driver2_time = driver2_tel['Time'].dt.total_seconds().values
driver1_time = driver1_time - driver1_time[0]
driver2_time = driver2_time - driver2_time[0]
print(f"{driver1}: {len(driver1_x)} data points")
print(f"{driver2}: {len(driver2_x)} data points")
gc.collect()


In [ ]:
# CREATE ANIMATION

print(f"\n{'='*60}")
print("Creating Animation...")
print(f"\n{'='*60}")

fig,ax = plt.subplots(figsize=(10,8),dpi=60)
fig.patch.set_facecolor('#1a1a1a')
ax.set_facecolor('#2a2a2a')

track_sample = 5
ax.plot(driver1_x[::track_sample],driver1_y[::track_sample],color='white',alpha=0.3,linewidth=1.5,linestyle='--',label='Track Layout')
driver1_point, = ax.plot([],[],'o',color='#0600ef',markersize=12,label=f'{driver1}',markeredgecolor='white',markeredgewidth=1.5)
driver2_point, = ax.plot([],[],'o',color='#dc0000',markersize=12,label=f'{driver2}',markeredgecolor='white',markeredgewidth=1.5)

driver1_trail, = ax.plot([],[],color='#0600ef',alpha=0.5,linewidth=1.5)
driver2_trail, = ax.plot([],[],color='#dc0000',alpha=0.5,linewidth=1.5)

time_text = ax.text(0.02,0.98,'',transform=ax.transAxes,fontsize=11,verticalalignment='top',bbox=dict(boxstyle='round',facecolor='black',alpha=0.8),color='white',fontfamily='monospace')
title_text = (f"{circuit_info['EventName']} - {circuit_info['Location']}\n"
              f"{driver1} vs {driver2} - Fastest Lap Comparison")
ax.set_title(title_text,fontsize=14,color='white',pad=15,fontweight='bold')

ax.set_xlabel('X Position (m)',fontsize=10,color='white')
ax.set_ylabel('Y Position (m)',fontsize=10,color='white')
ax.tick_params(colors='white',labelsize=9)
ax.legend(loc='upper right',fontsize=9,facecolor='black',edgecolor='white',labelcolor='white')
ax.grid(True,alpha=0.2,color='white')
ax.set_aspect('equal')

FPS = 15
max_time = max(driver1_time[-1],driver2_time[-1])
total_frames = int(max_time * FPS)

print("Animation Settings:")
print(f"FPS: {FPS}")
print(f"Total Frames: {total_frames}")
print(f"Duration: {max_time:.2f} seconds")

def get_position(time_val,time_array,x_array,y_array):
    if time_val > time_array[-1]:
        return x_array[-1],y_array[-1]
    idx = np.searchsorted(time_array,time_val)
    if idx == 0:
        return x_array[0],y_array[0]
    if idx >= len(time_array):
        return x_array[-1],y_array[-1]
    t1,t2 = time_array[idx-1],time_array[idx]
    alpha = (time_val - t1) / (t2-t1) if t2 != t1 else 0
    x = x_array[idx-1] + alpha * (x_array[idx] - x_array[idx-1])
    y = y_array[idx-1] + alpha * (y_array[idx] -  y_array[idx-1])
    return x, y

def animate(frame):
    current_time = frame / FPS
    x1,y1 = get_position(current_time,driver1_time,driver1_x,driver1_y)
    x2,y2 = get_position(current_time,driver2_time,driver2_x,driver2_y)
    driver1_point.set_data([x1],[y1])
    driver2_point.set_data([x2],[y2])

    trail_length = 30
    start_frame = max(0,frame - trail_length)
    trail1_x, trail1_y = [],[]
    trail2_x, trail2_y = [], []
    for f in range(start_frame,frame+1,2):
        t = f/FPS
        tx1,ty1 = get_position(t,driver1_time,driver1_x,driver1_y)
        tx2,ty2 = get_position(t,driver2_time,driver2_x,driver2_y)
        trail1_x.append(tx1)
        trail1_y.append(ty1)
        trail2_x.append(tx2)
        trail2_y.append(ty2)
    
    driver1_trail.set_data(trail1_x,trail1_y)
    driver2_trail.set_data(trail2_x,trail2_y)
    time_text.set_text(f'Time: {current_time:.2f}s\n'
                       f'{driver1}: {driver1_fastest["LapTime"]}\n'
                       f'{driver2}: {driver2_fastest["LapTime"]}')
    if frame%50 == 0:
        progress = (frame/total_frames) * 100
        print(f"Rendering: {progress:.1f}% Complete",end='\r')
    return driver1_point,driver2_point,driver1_trail,driver2_trail,time_text

print("\nBuilding animation frames (this is faster with optimizations)...")
anim = FuncAnimation(fig, animate, frames=total_frames,interval=1000/FPS, blit=True, repeat=True)


In [ ]:
#SAVE ANIMATION

output_file = f'{YEAR}_{RACE}_{driver1}_vs_{driver2}_lite.gif'
print(f"Saving animation to {output_file}")

writer = PillowWriter(fps=FPS,bitrate=1800)
anim.save(output_file,writer=writer,dpi=60)

print(f"\n{'='*60}")
print("Animation Saved Successfully")
print(f"File: {output_file}")
print(f"Duration: {max_time:.2f} seconds")
print(f"Frames: {total_frames}")
print(f"FPS: {FPS}")
print(f"{'='*60}")

plt.close('all')
gc.collect()

from IPython.display import Image, display
print("\nDisplaying Animation...")
display(Image(filename=output_file))

print("\nAnimation Complete")